# **GROUP ASSIGNMENT 2**

| Name      | UOW ID |
| ----------- | ----------- |
| Calaunan Alexander Jr Sumampong      | 7559161       |
| Deon Cham Hui Ern   | 7559471        |
| Elroy Chua Ming Xuan | 7431673 |
| Gonzales Raizel Vera Marie L. | 7436634 |

In [95]:
#imports 
import numpy as np
import pandas as pd
import pyspark as ps
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
# Initialising Spark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CSCI316 GA2')\
.config('spark-master', 'local')\
.getOrCreate()
spark

In [97]:
# Read the datasets
test_df = spark.read.csv('UNSW_NB15_testing-set.csv', header = True, inferSchema=True)
train_df = spark.read.csv('UNSW_NB15_training-set.csv', header = True, inferSchema=True)

test_df.show(5)

+---+--------+-----+-------+-----+-----+-----+------+------+---------+----+----+-----------+-----------+-----+-----+----------+----------+-----------+-----------+----+----------+----------+----+--------+--------+--------+-----+-----+-----------+-----------------+----------+------------+----------+----------------+----------------+--------------+------------+----------+----------------+----------+----------+---------------+----------+-----+
| id|     dur|proto|service|state|spkts|dpkts|sbytes|dbytes|     rate|sttl|dttl|      sload|      dload|sloss|dloss|    sinpkt|    dinpkt|       sjit|       djit|swin|     stcpb|     dtcpb|dwin|  tcprtt|  synack|  ackdat|smean|dmean|trans_depth|response_body_len|ct_srv_src|ct_state_ttl|ct_dst_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|is_ftp_login|ct_ftp_cmd|ct_flw_http_mthd|ct_src_ltm|ct_srv_dst|is_sm_ips_ports|attack_cat|label|
+---+--------+-----+-------+-----+-----+-----+------+------+---------+----+----+-----------+-----------+-----+--

In [98]:
# use pd to read csv
pd_train_df = pd.read_csv('UNSW_NB15_training-set.csv')
pd_test_df = pd.read_csv('UNSW_NB15_testing-set.csv')

# get info
pd_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [99]:
# get info
pd_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

## DROP UNNECESSARY COLUMNS


In [ ]:
# drop unnecessary columns
# drop id
pd_test_df = pd_test_df.drop(['id'], axis=1)

# ================================================
# ||&emsp;&emsp;&emsp;&emsp;&emsp;IM PLAYING AROUND TO DISCOVER DATA&emsp;&emsp;&emsp;&emsp;&emsp;||
# ================================================

In [100]:
# get values of the y columns:
pd_test_df['label'].value_counts()

label
1    119341
0     56000
Name: count, dtype: int64

In [101]:
pd_test_df['attack_cat'].value_counts()

attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64

In [102]:
# get values of object columns

print(pd_test_df['proto'].value_counts())

# use for loop to get values of object columns
for col in pd_test_df.columns:
    if pd_test_df[col].dtype == 'object':
        print(col, pd_test_df[col].unique())

proto
tcp       79946
udp       63283
unas      12084
arp        2859
ospf       2595
          ...  
rdp          98
netblt       98
igmp         18
icmp         15
rtp           1
Name: count, Length: 133, dtype: int64
proto ['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp'
 'wsn' 'pvp' 'wb-expak' 'mtp' 'pri-enc' 'sat-mon' 'cphb' 'sun-nd' 'iso-ip'
 'xtp' 'il' 'unas' 'mfe-nsp' '3pc' 'ipv6-route' 'idrp' 'bna' 'swipe'
 'kryptolan' 'cpnx' 'rsvp' 'wb-mon' 'vmtp' 'ib' 'dgp' 'eigrp' 'ax.25'
 'gmtp' 'pnni' 'sep' 'pgm' 'idpr-cmtp' 'zero' 'rvd' 'mobile' 'narp' 'fc'
 'pipe' 'ipcomp' 'ipv6-no' 'sat-expak' 'ipv6-opts' 'snp' 'ipcv'
 'br-sat-mon' 'ttp' 'tcf' 'nsfnet-igp' 'sprite-rpc' 'aes-sp3-d' 'sccopmce'
 'sctp' 'qnx' 'scps' 'etherip' 'aris' 'pim' 'compaq-peer' 'vrrp' 'iatp'
 'stp' 'l2tp' 'srp' 'sm' 'isis' 'smp' 'fire' 'ptp' 'crtp' 'sps'
 'merit-inp' 'idpr' 'skip' 'any' 'larp' 'ipip' 'micp' 'encap' 'ifmp'
 'tp++' 'a/n' 'ipv6' 'i-nlsp' 'ipx-n-ip' 'sdrp' 'tlsp' 'gre' 'mhrp' 'dd

In [103]:
# manually encode the column 'proto' from pd_test_df to a new column 'proto_num'

df_corr = pd_test_df.copy()

service_values = ['-', 'ftp', 'smtp', 'snmp', 'http', 'ftp-data', 'dns', 'ssh', 'radius', 'pop3',
 'dhcp', 'ssl', 'irc']

state_values = ['FIN', 'INT', 'CON', 'ECO', 'REQ', 'RST', 'PAR', 'URN', 'no']

attack_cat_values = ['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode', 'Reconnaissance',
 'Exploits', 'DoS', 'Worms', 'Generic']

proto_values = ['tcp', 'udp', 'arp', 'ospf', 'icmp', 'igmp', 'rtp', 'ddp', 'ipv6-frag', 'cftp',
               'wsn', 'pvp', 'wb-expak', 'mtp', 'pri-enc', 'sat-mon', 'cphb', 'sun-nd', 'iso-ip',
               'xtp', 'il', 'unas', 'mfe-nsp', '3pc', 'ipv6-route', 'idrp', 'bna', 'swipe',
               'kryptolan', 'cpnx', 'rsvp', 'wb-mon', 'vmtp', 'ib', 'dgp', 'eigrp', 'ax.25',
               'gmtp', 'pnni', 'sep', 'pgm', 'idpr-cmtp', 'zero', 'rvd', 'mobile', 'narp', 'fc',
               'pipe', 'ipcomp', 'ipv6-no', 'sat-expak', 'ipv6-opts', 'snp', 'ipcv',
               'br-sat-mon', 'ttp', 'tcf', 'nsfnet-igp', 'sprite-rpc', 'aes-sp3-d', 'sccopmce',
               'sctp', 'qnx', 'scps', 'etherip', 'aris', 'pim', 'compaq-peer', 'vrrp', 'iatp',
               'stp', 'l2tp', 'srp', 'sm', 'isis', 'smp', 'fire', 'ptp', 'crtp', 'sps',
               'merit-inp', 'idpr', 'skip', 'any', 'larp', 'ipip', 'micp', 'encap', 'ifmp',
               'tp++', 'a/n', 'ipv6', 'i-nlsp', 'ipx-n-ip', 'sdrp', 'tlsp', 'gre', 'mhrp', 'ddx',
               'ippc', 'visa', 'secure-vmtp', 'uti', 'vines', 'crudp', 'iplt', 'ggp', 'ip',
               'ipnip', 'st2', 'argus', 'bbn-rcc', 'egp', 'emcon', 'igp', 'nvp', 'pup', 'xnet',
               'chaos', 'mux', 'dcn', 'hmp', 'prm', 'trunk-1', 'xns-idp', 'leaf-1', 'leaf-2',
               'rdp', 'irtp', 'iso-tp4', 'netblt', 'trunk-2', 'cbt']

service_dict = {service_values[i]: i for i in range(len(service_values))}
proto_dict = {proto_values[i]: i for i in range(len(proto_values))}
state_dict = {state_values[i]: i for i in range(len(state_values))}
attack_cat_dict = {attack_cat_values[i]: i for i in range(len(attack_cat_values))}

df_corr['proto'] = pd_test_df['proto'].map(proto_dict)
df_corr['service'] = pd_test_df['service'].map(service_dict)
df_corr['state'] = pd_test_df['state'].map(state_dict)
df_corr['attack_cat'] = pd_test_df['attack_cat'].map(attack_cat_dict)

# drop the original columns
# df_corr.drop(['proto', 'service', 'state', 'attack_cat'], axis=1, inplace=True)

# =========================== SEE THINGS =========================== #
# df_corr.head()
# df_corr['service_num'].value_counts()
# df_corr['proto_num'].value_counts()
df_corr.info()
# ================================================================== #

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  int64  
 3   service            175341 non-null  int64  
 4   state              175341 non-null  int64  
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

## GET CORRELATION STUFF

In [106]:
from scipy.stats import pointbiserialr
import scipy.stats

# identify columns that are categorical
# identify columns that are numerical

# categorical columns
categorical_features = ['proto', 'service', 'state']

# continuous columns
continuous_features = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes','rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss','sinpkt', 'dinpkt', 'sjit', 
'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']

# target columns
target = ['label']

# # Compute Cramer's V correlations with the target attribute for categorical features
# Define Cramer's V function
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = scipy.stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Compute Cramer's V for all the categorical features
cramers_v_features = pd.Series(index=categorical_features)
for column in categorical_features:
    cramers_v_features[column] = cramers_v(df_corr[column], df_corr['label'])

# Compute point-biserial correlations with the target attribute for numerical features
point_biserial_correlations = pd.Series(index=continuous_features)
for column in continuous_features:
    point_biserial_correlations[column] = pointbiserialr(df_corr[column], df_corr['label'])[0]

# Sort the correlations in descending order
cramers_v_features_sorted = cramers_v_features.sort_values(ascending=False)
point_biserial_correlations_sorted = point_biserial_correlations.sort_values(ascending=False)

# Display the sorted results
print("\nPoint-Biserial Correlations (sorted):")
print(point_biserial_correlations_sorted)

# Display the sorted results
print("\nCramer's V Correlation (sorted):")
print(cramers_v_features_sorted)

# Combine the results into a single DataFrame
all_correlations = pd.concat([point_biserial_correlations_sorted, cramers_v_features_sorted])
all_correlations = all_correlations.sort_values(ascending=False)

print("\nAll Correlations (sorted):")
print(all_correlations)


Point-Biserial Correlations (sorted):
sttl                 0.692741
ct_state_ttl         0.577704
ct_dst_sport_ltm     0.357213
rate                 0.337979
ct_src_dport_ltm     0.305579
ct_dst_src_ltm       0.303855
ct_src_ltm           0.238225
ct_dst_ltm           0.229887
ct_srv_src           0.229044
ct_srv_dst           0.228046
sload                0.182870
ackdat               0.097364
dttl                 0.095049
tcprtt               0.081584
synack               0.058299
dur                  0.036175
sbytes               0.018576
ct_flw_http_mthd     0.015800
trans_depth          0.010801
sloss               -0.000640
sjit                -0.007069
smean               -0.010798
ct_ftp_cmd          -0.011055
is_ftp_login        -0.011055
response_body_len   -0.021361
dinpkt              -0.022887
spkts               -0.052178
djit                -0.060870
dbytes              -0.076871
dloss               -0.094685
dpkts               -0.118591
sinpkt              -0.176110
i

In [107]:
# now do correlation scores using the other target variable

# target columns
target = ['attack_cat']

# Compute Cramer's V for all the categorical features
cramers_v_features = pd.Series(index=categorical_features)
for column in categorical_features:
    cramers_v_features[column] = cramers_v(df_corr[column], df_corr['attack_cat'])

# Compute point-biserial correlations with the target attribute for numerical features
point_biserial_correlations = pd.Series(index=continuous_features)
for column in continuous_features:
    point_biserial_correlations[column] = pointbiserialr(df_corr[column], df_corr['attack_cat'])[0]

# Sort the correlations in descending order
cramers_v_features_sorted = cramers_v_features.sort_values(ascending=False)
point_biserial_correlations_sorted = point_biserial_correlations.sort_values(ascending=False)

# Display the sorted results
print("\nPoint-Biserial Correlations (sorted):")
print(point_biserial_correlations_sorted)

# Display the sorted results
print("\nCramer's V Correlation (sorted):")
print(cramers_v_features_sorted)

# Combine the results into a single DataFrame
all_correlations = pd.concat([point_biserial_correlations_sorted, cramers_v_features_sorted])
all_correlations = all_correlations.sort_values(ascending=False)

print("\nAll Correlations (sorted):")
print(all_correlations)


Point-Biserial Correlations (sorted):
ct_dst_sport_ltm     0.660850
sttl                 0.609699
ct_src_dport_ltm     0.589682
ct_dst_src_ltm       0.584426
ct_state_ttl         0.579500
ct_srv_dst           0.526023
ct_dst_ltm           0.520789
ct_srv_src           0.520611
ct_src_ltm           0.497272
rate                 0.424079
sload                0.143882
sbytes               0.009202
sloss               -0.009752
trans_depth         -0.012270
response_body_len   -0.016370
ct_ftp_cmd          -0.023093
is_ftp_login        -0.023093
dur                 -0.029297
ct_flw_http_mthd    -0.041587
dinpkt              -0.054474
spkts               -0.057996
sjit                -0.058481
dbytes              -0.065612
djit                -0.077286
ackdat              -0.082907
dloss               -0.083631
tcprtt              -0.105600
dpkts               -0.108928
synack              -0.115704
smean               -0.118813
dttl                -0.133221
sinpkt              -0.156242
i

# =================================================
# ||&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;OKAY IM DONE PLAYING&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;||
# =================================================